# Supervised Feature extraction Pipeline
This Notebook is extracting predefined features for a cohort

In [ ]:
# Import FIBER
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug, TobaccoUse, VitalSign, LabValue, Procedure
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 
from fiber.utils import Timer
import math

In [ ]:
#generic function get cohort 
def df_to_cohort(df):
    #mrns = list(df.index.values)
    #mrns = list(map(str, mrns))
    p_condition = MRNs(df)
    print(MRNs(df))
    cohort = Cohort(p_condition)
    #print(mrns)
    return cohort

In [ ]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW

def get_has_certain_condition(condition, df_mrn, name_feature, frequency):
    #get cohort
    cohort = df_to_cohort(df_mrn)
    #print(cohort)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    #print (cohort_condition)
    b=cohort_condition.empty
    print(b)
    if b==True:
        return cohort_condition
    cohort_condition=cohort_condition.astype({'medical_record_number': 'int64'})
    #print (cohort_condition)
    Filtered_DF = df_mrn.merge(cohort_condition, left_on="medical_record_number", right_on = "medical_record_number")
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
   
    Filtered_DF = Filtered_DF[["age_in_days", "HF_Onset_age_in_days"]]
    Filtered_DF = Filtered_DF.loc[Filtered_DF["age_in_days"] > Filtered_DF["HF_Onset_age_in_days"] ]
    #print(Filtered_DF)   

    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_on="medical_record_number", right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF["HF_Onset_age_in_days"]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={'HF_Onset_age_in_days':'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_on="medical_record_number", right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        for mrn in cohort_indexes: 
            x = get_mrn_has_certain_condition_WINDOW(mrn,cohort_condition,df_mrn.loc[mrn]["HF_Onset_age_in_days"]-gap_in_days)
            a = [mrn, x]
            cohort_mrn_diagnosis.append(a)
    else:
        print("function not available")
    #Saving Cohort as Parquet: 
    df_name= (name_feature+"_"+"_"+frequency)  
    df_final = cohort_mrn_diagnosis
    return df_final

In [ ]:
#Condtion Hospitalization
condition_inpatient=Encounter(description='%Inpatient%')

## Diagnosis

These were defined by the Notebook Create_supervised_Features

In [ ]:
condition_vascular_cognitive_impairment=( 
Diagnosis( "290%", "ICD-9")
|Diagnosis( "290.0%", "ICD-9")
|Diagnosis( "290.1%", "ICD-9")
|Diagnosis( "290.10%", "ICD-9")
|Diagnosis( "290.11%", "ICD-9")
|Diagnosis( "290.12%", "ICD-9")
|Diagnosis( "290.13%", "ICD-9")
|Diagnosis( "290.2%", "ICD-9")
|Diagnosis( "290.20%", "ICD-9")
|Diagnosis( "290.21%", "ICD-9")
|Diagnosis( "290.3%", "ICD-9")
|Diagnosis( "290.4%", "ICD-9")
|Diagnosis( "290.40%", "ICD-9")
|Diagnosis( "290.41%", "ICD-9")
|Diagnosis( "290.42%", "ICD-9")
|Diagnosis( "290.43%", "ICD-9")
|Diagnosis( "290.8%", "ICD-9")
|Diagnosis( "290.9%", "ICD-9")
|Diagnosis( "294.1%", "ICD-9")
|Diagnosis( "294.11%", "ICD-9")
|Diagnosis( "294.8%", "ICD-9")
|Diagnosis( "294.9%", "ICD-9")
|Diagnosis( "331%", "ICD-9")
|Diagnosis( "331.0%", "ICD-9")
|Diagnosis( "331.1%", "ICD-9")
|Diagnosis( "331.11%", "ICD-9")
|Diagnosis( "331.19%", "ICD-9")
|Diagnosis( "331.2%", "ICD-9")
|Diagnosis( "331.3%", "ICD-9")
|Diagnosis( "331.4%", "ICD-9")
|Diagnosis( "331.5%", "ICD-9")
|Diagnosis( "331.6%", "ICD-9")
|Diagnosis( "331.7%", "ICD-9")
|Diagnosis( "331.8%", "ICD-9")
|Diagnosis( "331.81%", "ICD-9")
|Diagnosis( "331.82%", "ICD-9")
|Diagnosis( "331.83%", "ICD-9")
|Diagnosis( "331.89%", "ICD-9")
|Diagnosis( "331.9%", "ICD-9")
|Diagnosis( "F01.50%", "ICD-10")
|Diagnosis( "F01.51%", "ICD-10")
|Diagnosis( "F02.81%", "ICD-10")
|Diagnosis( "F03.90%", "ICD-10")
|Diagnosis( "F06.8%", "ICD-10")
|Diagnosis( "G30.9%", "ICD-10")
|Diagnosis( "G31.01%", "ICD-10")
|Diagnosis( "G31.09%", "ICD-10")
|Diagnosis( "G31.1%", "ICD-10")
|Diagnosis( "G31.83%", "ICD-10")
|Diagnosis( "G31.89%", "ICD-10")
|Diagnosis( "G31.9%", "ICD-10")
|Diagnosis( "G94%", "ICD-10")) 
condition_acute_myocardial_infarction=( 
Diagnosis( "410.00%", "ICD-9")
|Diagnosis( "410.01%", "ICD-9")
|Diagnosis( "410.02%", "ICD-9")
|Diagnosis( "410.10%", "ICD-9")
|Diagnosis( "410.11%", "ICD-9")
|Diagnosis( "410.12%", "ICD-9")
|Diagnosis( "410.20%", "ICD-9")
|Diagnosis( "410.21%", "ICD-9")
|Diagnosis( "410.22%", "ICD-9")
|Diagnosis( "410.30%", "ICD-9")
|Diagnosis( "410.31%", "ICD-9")
|Diagnosis( "410.32%", "ICD-9")
|Diagnosis( "410.40%", "ICD-9")
|Diagnosis( "410.41%", "ICD-9")
|Diagnosis( "410.42%", "ICD-9")
|Diagnosis( "410.50%", "ICD-9")
|Diagnosis( "410.51%", "ICD-9")
|Diagnosis( "410.52%", "ICD-9")
|Diagnosis( "410.80%", "ICD-9")
|Diagnosis( "410.81%", "ICD-9")
|Diagnosis( "410.82%", "ICD-9")
|Diagnosis( "410.90%", "ICD-9")
|Diagnosis( "410.91%", "ICD-9")
|Diagnosis( "410.92%", "ICD-9")
|Diagnosis( "I21.09%", "ICD-10")
|Diagnosis( "I21.11%", "ICD-10")
|Diagnosis( "I21.19%", "ICD-10")
|Diagnosis( "I21.29%", "ICD-10")
|Diagnosis( "I21.3%", "ICD-10")) 
condition_anemia=( 
Diagnosis( "280.9%", "ICD-9")
|Diagnosis( "281.1%", "ICD-9")
|Diagnosis( "281.2%", "ICD-9")
|Diagnosis( "281.3%", "ICD-9")
|Diagnosis( "281.4%", "ICD-9")
|Diagnosis( "281.9%", "ICD-9")
|Diagnosis( "283.10%", "ICD-9")
|Diagnosis( "283.11%", "ICD-9")
|Diagnosis( "283.9%", "ICD-9")
|Diagnosis( "284.01%", "ICD-9")
|Diagnosis( "284.2%", "ICD-9")
|Diagnosis( "284.81%", "ICD-9")
|Diagnosis( "284.89%", "ICD-9")
|Diagnosis( "284.9%", "ICD-9")
|Diagnosis( "285.21%", "ICD-9")
|Diagnosis( "285.22%", "ICD-9")
|Diagnosis( "285.29%", "ICD-9")
|Diagnosis( "285.9%", "ICD-9")
|Diagnosis( "D50.9%", "ICD-10")
|Diagnosis( "D64.9%", "ICD-10")) 
condition_angina=( 
Diagnosis( "413.0%", "ICD-9")
|Diagnosis( "413.1%", "ICD-9")
|Diagnosis( "413.9%", "ICD-9")
|Diagnosis( "I20%", "ICD-10")
|Diagnosis( "I20.0%", "ICD-10")
|Diagnosis( "I20.1%", "ICD-10")
|Diagnosis( "I20.8%", "ICD-10")
|Diagnosis( "I20.9%", "ICD-10")) 
condition_atrial_flutter_fibrillation=( 
Diagnosis( "427.31%", "ICD-9")
|Diagnosis( "427.32%", "ICD-9")
|Diagnosis( "I48.0%", "ICD-10")
|Diagnosis( "I48.1%", "ICD-10")
|Diagnosis( "I48.2%", "ICD-10")
|Diagnosis( "I48.3%", "ICD-10")
|Diagnosis( "I48.4%", "ICD-10")
|Diagnosis( "I48.91%", "ICD-10")
|Diagnosis( "I48.92%", "ICD-10")) 
condition_cardiomyopathy=( 
Diagnosis( "425%", "ICD-9")
|Diagnosis( "425.11%", "ICD-9")
|Diagnosis( "425.18%", "ICD-9")
|Diagnosis( "I42.1%", "ICD-10")
|Diagnosis( "I42.2%", "ICD-10")
|Diagnosis( "I42.7%", "ICD-10")
|Diagnosis( "I42.8%", "ICD-10")) 
condition_chronic_kidney_disease=( 
Diagnosis( "585.1%", "ICD-9")
|Diagnosis( "585.2%", "ICD-9")
|Diagnosis( "585.3%", "ICD-9")
|Diagnosis( "585.4%", "ICD-9")
|Diagnosis( "585.5%", "ICD-9")
|Diagnosis( "585.9%", "ICD-9")
|Diagnosis( "N18.1%", "ICD-10")
|Diagnosis( "N18.2%", "ICD-10")
|Diagnosis( "N18.3%", "ICD-10")
|Diagnosis( "N18.4%", "ICD-10")
|Diagnosis( "N18.5%", "ICD-10")
|Diagnosis( "N18.9%", "ICD-10")) 
condition_COPD=( 
Diagnosis( "491.20%", "ICD-9")
|Diagnosis( "491.21%", "ICD-9")
|Diagnosis( "491.22%", "ICD-9")
|Diagnosis( "493.20%", "ICD-9")
|Diagnosis( "493.21%", "ICD-9")
|Diagnosis( "493.22%", "ICD-9")
|Diagnosis( "496%", "ICD-9")
|Diagnosis( "J44%", "ICD-10")
|Diagnosis( "J44.0%", "ICD-10")
|Diagnosis( "J44.1%", "ICD-10")
|Diagnosis( "J44.9%", "ICD-10")) 
condition_coronary_artery_disease=( 
Diagnosis( "410%", "ICD-9")
|Diagnosis( "410.0%", "ICD-9")
|Diagnosis( "410.00%", "ICD-9")
|Diagnosis( "410.01%", "ICD-9")
|Diagnosis( "410.02%", "ICD-9")
|Diagnosis( "410.1%", "ICD-9")
|Diagnosis( "410.10%", "ICD-9")
|Diagnosis( "410.11%", "ICD-9")
|Diagnosis( "410.12%", "ICD-9")
|Diagnosis( "410.2%", "ICD-9")
|Diagnosis( "410.20%", "ICD-9")
|Diagnosis( "410.21%", "ICD-9")
|Diagnosis( "410.22%", "ICD-9")
|Diagnosis( "410.3%", "ICD-9")
|Diagnosis( "410.30%", "ICD-9")
|Diagnosis( "410.31%", "ICD-9")
|Diagnosis( "410.32%", "ICD-9")
|Diagnosis( "410.4%", "ICD-9")
|Diagnosis( "410.40%", "ICD-9")
|Diagnosis( "410.41%", "ICD-9")
|Diagnosis( "410.42%", "ICD-9")
|Diagnosis( "410.5%", "ICD-9")
|Diagnosis( "410.50%", "ICD-9")
|Diagnosis( "410.51%", "ICD-9")
|Diagnosis( "410.52%", "ICD-9")
|Diagnosis( "410.6%", "ICD-9")
|Diagnosis( "410.60%", "ICD-9")
|Diagnosis( "410.61%", "ICD-9")
|Diagnosis( "410.62%", "ICD-9")
|Diagnosis( "410.7%", "ICD-9")
|Diagnosis( "410.70%", "ICD-9")
|Diagnosis( "410.71%", "ICD-9")
|Diagnosis( "410.72%", "ICD-9")
|Diagnosis( "410.8%", "ICD-9")
|Diagnosis( "410.80%", "ICD-9")
|Diagnosis( "410.81%", "ICD-9")
|Diagnosis( "410.82%", "ICD-9")
|Diagnosis( "410.9%", "ICD-9")
|Diagnosis( "410.90%", "ICD-9")
|Diagnosis( "410.91%", "ICD-9")
|Diagnosis( "410.92%", "ICD-9")
|Diagnosis( "414.00%", "ICD-9")
|Diagnosis( "414.01%", "ICD-9")
|Diagnosis( "414.02%", "ICD-9")
|Diagnosis( "414.03%", "ICD-9")
|Diagnosis( "414.04%", "ICD-9")
|Diagnosis( "414.05%", "ICD-9")
|Diagnosis( "414.06%", "ICD-9")
|Diagnosis( "414.07%", "ICD-9")
|Diagnosis( "414.2%", "ICD-9")
|Diagnosis( "414.3%", "ICD-9")
|Diagnosis( "414.8%", "ICD-9")
|Diagnosis( "414.9%", "ICD-9")
|Diagnosis( "V45.81%", "ICD-9")
|Diagnosis( "V45.82%", "ICD-9")
|Diagnosis( "I21.09%", "ICD-10")
|Diagnosis( "I21.11%", "ICD-10")
|Diagnosis( "I21.19%", "ICD-10")
|Diagnosis( "I21.29%", "ICD-10")
|Diagnosis( "I21.3%", "ICD-10")
|Diagnosis( "I21.4%", "ICD-10")
|Diagnosis( "I21.9%", "ICD-10")
|Diagnosis( "I21.A1%", "ICD-10")
|Diagnosis( "I21.A9%", "ICD-10")
|Diagnosis( "I25.1%", "ICD-10")
|Diagnosis( "I25.10%", "ICD-10")
|Diagnosis( "I25.11%", "ICD-10")
|Diagnosis( "I25.110%", "ICD-10")
|Diagnosis( "I25.111%", "ICD-10")
|Diagnosis( "I25.118%", "ICD-10")
|Diagnosis( "I25.119%", "ICD-10")
|Diagnosis( "I25.5%", "ICD-10")
|Diagnosis( "I25.810%", "ICD-10")
|Diagnosis( "I25.811%", "ICD-10")
|Diagnosis( "I25.812%", "ICD-10")
|Diagnosis( "I25.82%", "ICD-10")
|Diagnosis( "I25.83%", "ICD-10")
|Diagnosis( "I25.89%", "ICD-10")
|Diagnosis( "I25.9%", "ICD-10")
|Diagnosis( "Z95.1%", "ICD-10")
|Diagnosis( "Z98.61%", "ICD-10")) 
condition_depression=( 
Diagnosis( "296.20%", "ICD-9")
|Diagnosis( "296.21%", "ICD-9")
|Diagnosis( "296.22%", "ICD-9")
|Diagnosis( "296.23%", "ICD-9")
|Diagnosis( "296.24%", "ICD-9")
|Diagnosis( "296.25%", "ICD-9")
|Diagnosis( "296.26%", "ICD-9")
|Diagnosis( "296.30%", "ICD-9")
|Diagnosis( "296.31%", "ICD-9")
|Diagnosis( "296.32%", "ICD-9")
|Diagnosis( "296.33%", "ICD-9")
|Diagnosis( "296.34%", "ICD-9")
|Diagnosis( "296.35%", "ICD-9")
|Diagnosis( "296.36%", "ICD-9")
|Diagnosis( "300.0%", "ICD-9")
|Diagnosis( "300.00%", "ICD-9")
|Diagnosis( "300.01%", "ICD-9")
|Diagnosis( "300.02%", "ICD-9")
|Diagnosis( "300.09%", "ICD-9")
|Diagnosis( "300.1%", "ICD-9")
|Diagnosis( "300.10%", "ICD-9")
|Diagnosis( "300.11%", "ICD-9")
|Diagnosis( "300.12%", "ICD-9")
|Diagnosis( "300.13%", "ICD-9")
|Diagnosis( "300.14%", "ICD-9")
|Diagnosis( "300.15%", "ICD-9")
|Diagnosis( "300.16%", "ICD-9")
|Diagnosis( "300.19%", "ICD-9")
|Diagnosis( "300.2%", "ICD-9")
|Diagnosis( "300.20%", "ICD-9")
|Diagnosis( "300.21%", "ICD-9")
|Diagnosis( "300.22%", "ICD-9")
|Diagnosis( "300.23%", "ICD-9")
|Diagnosis( "300.29%", "ICD-9")
|Diagnosis( "300.3%", "ICD-9")
|Diagnosis( "300.4%", "ICD-9")
|Diagnosis( "300.5%", "ICD-9")
|Diagnosis( "300.6%", "ICD-9")
|Diagnosis( "300.7%", "ICD-9")
|Diagnosis( "300.8%", "ICD-9")
|Diagnosis( "300.81%", "ICD-9")
|Diagnosis( "300.82%", "ICD-9")
|Diagnosis( "300.89%", "ICD-9")
|Diagnosis( "300.9%", "ICD-9")
|Diagnosis( "309.0%", "ICD-9")
|Diagnosis( "309.1%", "ICD-9")
|Diagnosis( "309.2%", "ICD-9")
|Diagnosis( "309.21%", "ICD-9")
|Diagnosis( "309.22%", "ICD-9")
|Diagnosis( "309.23%", "ICD-9")
|Diagnosis( "309.24%", "ICD-9")
|Diagnosis( "309.28%", "ICD-9")
|Diagnosis( "309.29%", "ICD-9")
|Diagnosis( "309.3%", "ICD-9")
|Diagnosis( "309.4%", "ICD-9")
|Diagnosis( "309.8%", "ICD-9")
|Diagnosis( "309.81%", "ICD-9")
|Diagnosis( "309.82%", "ICD-9")
|Diagnosis( "309.83%", "ICD-9")
|Diagnosis( "309.89%", "ICD-9")
|Diagnosis( "309.9%", "ICD-9")
|Diagnosis( "311%", "ICD-9")
|Diagnosis( "F32.0%", "ICD-10")
|Diagnosis( "F32.1%", "ICD-10")
|Diagnosis( "F32.2%", "ICD-10")
|Diagnosis( "F32.3%", "ICD-10")
|Diagnosis( "F32.4%", "ICD-10")
|Diagnosis( "F32.5%", "ICD-10")
|Diagnosis( "F32.9%", "ICD-10")
|Diagnosis( "F33.0%", "ICD-10")
|Diagnosis( "F33.1%", "ICD-10")
|Diagnosis( "F33.2%", "ICD-10")
|Diagnosis( "F33.3%", "ICD-10")
|Diagnosis( "F33.41%", "ICD-10")
|Diagnosis( "F33.42%", "ICD-10")
|Diagnosis( "F33.9%", "ICD-10")
|Diagnosis( "F34.1%", "ICD-10")
|Diagnosis( "F40.01%", "ICD-10")
|Diagnosis( "F40.02%", "ICD-10")
|Diagnosis( "F40.10%", "ICD-10")
|Diagnosis( "F40.218%", "ICD-10")
|Diagnosis( "F40.240%", "ICD-10")
|Diagnosis( "F40.241%", "ICD-10")
|Diagnosis( "F40.8%", "ICD-10")
|Diagnosis( "F40.9%", "ICD-10")
|Diagnosis( "F41.0%", "ICD-10")
|Diagnosis( "F41.1%", "ICD-10")
|Diagnosis( "F41.8%", "ICD-10")
|Diagnosis( "F41.9%", "ICD-10")
|Diagnosis( "F42.2%", "ICD-10")
|Diagnosis( "F42.3%", "ICD-10")
|Diagnosis( "F42.8%", "ICD-10")
|Diagnosis( "F42.9%", "ICD-10")
|Diagnosis( "F43.10%", "ICD-10")
|Diagnosis( "F43.12%", "ICD-10")
|Diagnosis( "F43.20%", "ICD-10")
|Diagnosis( "F43.21%", "ICD-10")
|Diagnosis( "F43.22%", "ICD-10")
|Diagnosis( "F43.23%", "ICD-10")
|Diagnosis( "F43.24%", "ICD-10")
|Diagnosis( "F43.25%", "ICD-10")
|Diagnosis( "F43.29%", "ICD-10")
|Diagnosis( "F43.8%", "ICD-10")
|Diagnosis( "F44.0%", "ICD-10")
|Diagnosis( "F44.1%", "ICD-10")
|Diagnosis( "F44.4%", "ICD-10")
|Diagnosis( "F44.6%", "ICD-10")
|Diagnosis( "F44.81%", "ICD-10")
|Diagnosis( "F44.89%", "ICD-10")
|Diagnosis( "F44.9%", "ICD-10")
|Diagnosis( "F45.0%", "ICD-10")
|Diagnosis( "F45.1%", "ICD-10")
|Diagnosis( "F45.21%", "ICD-10")
|Diagnosis( "F45.22%", "ICD-10")
|Diagnosis( "F45.8%", "ICD-10")
|Diagnosis( "F45.9%", "ICD-10")
|Diagnosis( "F48.1%", "ICD-10")
|Diagnosis( "F48.8%", "ICD-10")
|Diagnosis( "F48.9%", "ICD-10")
|Diagnosis( "F68.11%", "ICD-10")
|Diagnosis( "F68.8%", "ICD-10")
|Diagnosis( "F93.0%", "ICD-10")
|Diagnosis( "F94.8%", "ICD-10")
|Diagnosis( "F99%", "ICD-10")) 
condition_diabetes_mellitus_type_I=( 
Diagnosis( "250.01%", "ICD-9")
|Diagnosis( "250.03%", "ICD-9")
|Diagnosis( "250.11%", "ICD-9")
|Diagnosis( "250.13%", "ICD-9")
|Diagnosis( "250.23%", "ICD-9")
|Diagnosis( "250.31%", "ICD-9")
|Diagnosis( "250.33%", "ICD-9")
|Diagnosis( "250.41%", "ICD-9")
|Diagnosis( "250.43%", "ICD-9")
|Diagnosis( "250.51%", "ICD-9")
|Diagnosis( "250.53%", "ICD-9")
|Diagnosis( "250.61%", "ICD-9")
|Diagnosis( "250.63%", "ICD-9")
|Diagnosis( "250.71%", "ICD-9")
|Diagnosis( "250.73%", "ICD-9")
|Diagnosis( "250.81%", "ICD-9")
|Diagnosis( "250.83%", "ICD-9")
|Diagnosis( "250.93%", "ICD-9")
|Diagnosis( "E10%", "ICD-10")
|Diagnosis( "E10.1%", "ICD-10")
|Diagnosis( "E10.10%", "ICD-10")
|Diagnosis( "E10.11%", "ICD-10")
|Diagnosis( "E10.2%", "ICD-10")
|Diagnosis( "E10.21%", "ICD-10")
|Diagnosis( "E10.22%", "ICD-10")
|Diagnosis( "E10.29%", "ICD-10")
|Diagnosis( "E10.3%", "ICD-10")
|Diagnosis( "E10.31%", "ICD-10")
|Diagnosis( "E10.311%", "ICD-10")
|Diagnosis( "E10.319%", "ICD-10")
|Diagnosis( "E10.32%", "ICD-10")
|Diagnosis( "E10.321%", "ICD-10")
|Diagnosis( "E10.329%", "ICD-10")
|Diagnosis( "E10.33%", "ICD-10")
|Diagnosis( "E10.331%", "ICD-10")
|Diagnosis( "E10.339%", "ICD-10")
|Diagnosis( "E10.34%", "ICD-10")
|Diagnosis( "E10.341%", "ICD-10")
|Diagnosis( "E10.349%", "ICD-10")
|Diagnosis( "E10.35%", "ICD-10")
|Diagnosis( "E10.351%", "ICD-10")
|Diagnosis( "E10.359%", "ICD-10")
|Diagnosis( "E10.36%", "ICD-10")
|Diagnosis( "E10.39%", "ICD-10")
|Diagnosis( "E10.4%", "ICD-10")
|Diagnosis( "E10.40%", "ICD-10")
|Diagnosis( "E10.41%", "ICD-10")
|Diagnosis( "E10.42%", "ICD-10")
|Diagnosis( "E10.43%", "ICD-10")
|Diagnosis( "E10.44%", "ICD-10")
|Diagnosis( "E10.49%", "ICD-10")
|Diagnosis( "E10.5%", "ICD-10")
|Diagnosis( "E10.51%", "ICD-10")
|Diagnosis( "E10.52%", "ICD-10")
|Diagnosis( "E10.59%", "ICD-10")
|Diagnosis( "E10.6%", "ICD-10")
|Diagnosis( "E10.61%", "ICD-10")
|Diagnosis( "E10.610%", "ICD-10")
|Diagnosis( "E10.618%", "ICD-10")
|Diagnosis( "E10.62%", "ICD-10")
|Diagnosis( "E10.620%", "ICD-10")
|Diagnosis( "E10.621%", "ICD-10")
|Diagnosis( "E10.622%", "ICD-10")
|Diagnosis( "E10.628%", "ICD-10")
|Diagnosis( "E10.63%", "ICD-10")
|Diagnosis( "E10.630%", "ICD-10")
|Diagnosis( "E10.638%", "ICD-10")
|Diagnosis( "E10.64%", "ICD-10")
|Diagnosis( "E10.641%", "ICD-10")
|Diagnosis( "E10.649%", "ICD-10")
|Diagnosis( "E10.65%", "ICD-10")
|Diagnosis( "E10.69%", "ICD-10")
|Diagnosis( "E10.8%", "ICD-10")
|Diagnosis( "E10.9%", "ICD-10")) 
condition_diabetes_mellitus_type_II=( 
Diagnosis( "250.00%", "ICD-9")
|Diagnosis( "250.02%", "ICD-9")
|Diagnosis( "250.12%", "ICD-9")
|Diagnosis( "250.20%", "ICD-9")
|Diagnosis( "250.22%", "ICD-9")
|Diagnosis( "250.30%", "ICD-9")
|Diagnosis( "250.32%", "ICD-9")
|Diagnosis( "250.40%", "ICD-9")
|Diagnosis( "250.42%", "ICD-9")
|Diagnosis( "250.50%", "ICD-9")
|Diagnosis( "250.52%", "ICD-9")
|Diagnosis( "250.60%", "ICD-9")
|Diagnosis( "250.62%", "ICD-9")
|Diagnosis( "250.70%", "ICD-9")
|Diagnosis( "250.72%", "ICD-9")
|Diagnosis( "250.80%", "ICD-9")
|Diagnosis( "250.82%", "ICD-9")
|Diagnosis( "250.90%", "ICD-9")
|Diagnosis( "250.92%", "ICD-9")
|Diagnosis( "E11.0%", "ICD-10")
|Diagnosis( "E11.00%", "ICD-10")
|Diagnosis( "E11.01%", "ICD-10")
|Diagnosis( "E11.10%", "ICD-10")
|Diagnosis( "E11.11%", "ICD-10")
|Diagnosis( "E11.2%", "ICD-10")
|Diagnosis( "E11.21%", "ICD-10")
|Diagnosis( "E11.22%", "ICD-10")
|Diagnosis( "E11.29%", "ICD-10")
|Diagnosis( "E11.3%", "ICD-10")
|Diagnosis( "E11.31%", "ICD-10")
|Diagnosis( "E11.311%", "ICD-10")
|Diagnosis( "E11.319%", "ICD-10")
|Diagnosis( "E11.32%", "ICD-10")
|Diagnosis( "E11.321%", "ICD-10")
|Diagnosis( "E11.329%", "ICD-10")
|Diagnosis( "E11.33%", "ICD-10")
|Diagnosis( "E11.331%", "ICD-10")
|Diagnosis( "E11.339%", "ICD-10")
|Diagnosis( "E11.34%", "ICD-10")
|Diagnosis( "E11.341%", "ICD-10")
|Diagnosis( "E11.349%", "ICD-10")
|Diagnosis( "E11.35%", "ICD-10")
|Diagnosis( "E11.351%", "ICD-10")
|Diagnosis( "E11.359%", "ICD-10")
|Diagnosis( "E11.36%", "ICD-10")
|Diagnosis( "E11.39%", "ICD-10")
|Diagnosis( "E11.4%", "ICD-10")
|Diagnosis( "E11.40%", "ICD-10")
|Diagnosis( "E11.41%", "ICD-10")
|Diagnosis( "E11.42%", "ICD-10")
|Diagnosis( "E11.43%", "ICD-10")
|Diagnosis( "E11.44%", "ICD-10")
|Diagnosis( "E11.49%", "ICD-10")
|Diagnosis( "E11.5%", "ICD-10")
|Diagnosis( "E11.51%", "ICD-10")
|Diagnosis( "E11.52%", "ICD-10")
|Diagnosis( "E11.59%", "ICD-10")
|Diagnosis( "E11.6%", "ICD-10")
|Diagnosis( "E11.61%", "ICD-10")
|Diagnosis( "E11.610%", "ICD-10")
|Diagnosis( "E11.618%", "ICD-10")
|Diagnosis( "E11.62%", "ICD-10")
|Diagnosis( "E11.620%", "ICD-10")
|Diagnosis( "E11.621%", "ICD-10")
|Diagnosis( "E11.622%", "ICD-10")
|Diagnosis( "E11.628%", "ICD-10")
|Diagnosis( "E11.63%", "ICD-10")
|Diagnosis( "E11.630%", "ICD-10")
|Diagnosis( "E11.638%", "ICD-10")
|Diagnosis( "E11.64%", "ICD-10")
|Diagnosis( "E11.641%", "ICD-10")
|Diagnosis( "E11.649%", "ICD-10")
|Diagnosis( "E11.65%", "ICD-10")
|Diagnosis( "E11.69%", "ICD-10")
|Diagnosis( "E11.8%", "ICD-10")
|Diagnosis( "E11.9%", "ICD-10")) 
condition_diabetic_nephropathy=( 
Diagnosis( "250.40%", "ICD-9")
|Diagnosis( "250.42%", "ICD-9")
|Diagnosis( "E10.21%", "ICD-10")
|Diagnosis( "E11.21%", "ICD-10")) 
condition_dyspnea=( 
Diagnosis( "786.0%", "ICD-9")
|Diagnosis( "786.0%", "ICD-9")
|Diagnosis( "786.01%", "ICD-9")
|Diagnosis( "786.02%", "ICD-9")
|Diagnosis( "786.03%", "ICD-9")
|Diagnosis( "786.04%", "ICD-9")
|Diagnosis( "786.05%", "ICD-9")
|Diagnosis( "786.06%", "ICD-9")
|Diagnosis( "786.07%", "ICD-9")
|Diagnosis( "786.09%", "ICD-9")) 
condition_dysrhythmias=( 
Diagnosis( "426%", "ICD-9")
|Diagnosis( "426.0%", "ICD-9")
|Diagnosis( "426.1%", "ICD-9")
|Diagnosis( "426.10%", "ICD-9")
|Diagnosis( "426.11%", "ICD-9")
|Diagnosis( "426.12%", "ICD-9")
|Diagnosis( "426.13%", "ICD-9")
|Diagnosis( "426.2%", "ICD-9")
|Diagnosis( "426.3%", "ICD-9")
|Diagnosis( "426.4%", "ICD-9")
|Diagnosis( "426.5%", "ICD-9")
|Diagnosis( "426.50%", "ICD-9")
|Diagnosis( "426.51%", "ICD-9")
|Diagnosis( "426.52%", "ICD-9")
|Diagnosis( "426.53%", "ICD-9")
|Diagnosis( "426.54%", "ICD-9")
|Diagnosis( "426.6%", "ICD-9")
|Diagnosis( "426.7%", "ICD-9")
|Diagnosis( "426.8%", "ICD-9")
|Diagnosis( "426.81%", "ICD-9")
|Diagnosis( "426.82%", "ICD-9")
|Diagnosis( "426.89%", "ICD-9")
|Diagnosis( "426.9%", "ICD-9")
|Diagnosis( "427%", "ICD-9")
|Diagnosis( "427.0%", "ICD-9")
|Diagnosis( "427.1%", "ICD-9")
|Diagnosis( "427.2%", "ICD-9")
|Diagnosis( "427.3%", "ICD-9")
|Diagnosis( "427.31%", "ICD-9")
|Diagnosis( "427.32%", "ICD-9")
|Diagnosis( "427.4%", "ICD-9")
|Diagnosis( "427.41%", "ICD-9")
|Diagnosis( "427.42%", "ICD-9")
|Diagnosis( "427.5%", "ICD-9")
|Diagnosis( "427.6%", "ICD-9")
|Diagnosis( "427.60%", "ICD-9")
|Diagnosis( "427.61%", "ICD-9")
|Diagnosis( "427.69%", "ICD-9")
|Diagnosis( "427.8%", "ICD-9")
|Diagnosis( "427.81%", "ICD-9")
|Diagnosis( "427.89%", "ICD-9")
|Diagnosis( "427.9%", "ICD-9")
|Diagnosis( "I44.0%", "ICD-10")
|Diagnosis( "I44.1%", "ICD-10")
|Diagnosis( "I44.2%", "ICD-10")
|Diagnosis( "I44.30%", "ICD-10")
|Diagnosis( "I44.39%", "ICD-10")
|Diagnosis( "I44.4%", "ICD-10")
|Diagnosis( "I44.5%", "ICD-10")
|Diagnosis( "I44.60%", "ICD-10")
|Diagnosis( "I44.69%", "ICD-10")
|Diagnosis( "I44.7%", "ICD-10")
|Diagnosis( "I45.10%", "ICD-10")
|Diagnosis( "I45.2%", "ICD-10")
|Diagnosis( "I45.3%", "ICD-10")
|Diagnosis( "I45.4%", "ICD-10")
|Diagnosis( "I45.5%", "ICD-10")
|Diagnosis( "I45.6%", "ICD-10")
|Diagnosis( "I45.81%", "ICD-10")
|Diagnosis( "I45.89%", "ICD-10")
|Diagnosis( "I45.9%", "ICD-10")
|Diagnosis( "I46.9%", "ICD-10")
|Diagnosis( "I47.0%", "ICD-10")
|Diagnosis( "I47.1%", "ICD-10")
|Diagnosis( "I47.2%", "ICD-10")
|Diagnosis( "I47.9%", "ICD-10")
|Diagnosis( "I48.91%", "ICD-10")
|Diagnosis( "I48.92%", "ICD-10")
|Diagnosis( "I49.01%", "ICD-10")
|Diagnosis( "I49.02%", "ICD-10")
|Diagnosis( "I49.1%", "ICD-10")
|Diagnosis( "I49.3%", "ICD-10")
|Diagnosis( "I49.40%", "ICD-10")
|Diagnosis( "I49.49%", "ICD-10")
|Diagnosis( "I49.5%", "ICD-10")
|Diagnosis( "I49.8%", "ICD-10")
|Diagnosis( "I49.9%", "ICD-10")) 
condition_edema=( 
Diagnosis( "782.3%", "ICD-9")
|Diagnosis( "R60.0%", "ICD-10")
|Diagnosis( "R60.1%", "ICD-10")
|Diagnosis( "R60.9%", "ICD-10")) 
condition_hyperkalemia=( 
Diagnosis( "276.7%", "ICD-9")
|Diagnosis( "E87.5%", "ICD-10")) 
condition_hyperlipidemia=( 
Diagnosis( "272.0%", "ICD-9")
|Diagnosis( "272.1%", "ICD-9")
|Diagnosis( "272.2%", "ICD-9")
|Diagnosis( "272.3%", "ICD-9")
|Diagnosis( "272.4%", "ICD-9")
|Diagnosis( "272.5%", "ICD-9")
|Diagnosis( "272.6%", "ICD-9")
|Diagnosis( "272.7%", "ICD-9")
|Diagnosis( "272.8%", "ICD-9")
|Diagnosis( "272.9%", "ICD-9")
|Diagnosis( "E75.21%", "ICD-10")
|Diagnosis( "E75.22%", "ICD-10")
|Diagnosis( "E75.249%", "ICD-10")
|Diagnosis( "E77.0%", "ICD-10")
|Diagnosis( "E77.1%", "ICD-10")
|Diagnosis( "E78.00%", "ICD-10")
|Diagnosis( "E78.01%", "ICD-10")
|Diagnosis( "E78.1%", "ICD-10")
|Diagnosis( "E78.2%", "ICD-10")
|Diagnosis( "E78.3%", "ICD-10")
|Diagnosis( "E78.4%", "ICD-10")
|Diagnosis( "E78.5%", "ICD-10")
|Diagnosis( "E78.6%", "ICD-10")
|Diagnosis( "E78.81%", "ICD-10")
|Diagnosis( "E78.89%", "ICD-10")
|Diagnosis( "E78.9%", "ICD-10")
|Diagnosis( "E88.1%", "ICD-10")
|Diagnosis( "E88.89%", "ICD-10")) 
condition_hypertension=( 
Diagnosis( "362.11%", "ICD-9")
|Diagnosis( "401%", "ICD-9")
|Diagnosis( "401.0%", "ICD-9")
|Diagnosis( "401.1%", "ICD-9")
|Diagnosis( "401.9%", "ICD-9")
|Diagnosis( "402%", "ICD-9")
|Diagnosis( "402.0%", "ICD-9")
|Diagnosis( "402.00%", "ICD-9")
|Diagnosis( "402.01%", "ICD-9")
|Diagnosis( "402.1%", "ICD-9")
|Diagnosis( "402.10%", "ICD-9")
|Diagnosis( "402.11%", "ICD-9")
|Diagnosis( "402.9%", "ICD-9")
|Diagnosis( "402.90%", "ICD-9")
|Diagnosis( "402.91%", "ICD-9")
|Diagnosis( "403%", "ICD-9")
|Diagnosis( "403.0%", "ICD-9")
|Diagnosis( "403.00%", "ICD-9")
|Diagnosis( "403.01%", "ICD-9")
|Diagnosis( "403.1%", "ICD-9")
|Diagnosis( "403.10%", "ICD-9")
|Diagnosis( "403.11%", "ICD-9")
|Diagnosis( "403.9%", "ICD-9")
|Diagnosis( "403.90%", "ICD-9")
|Diagnosis( "403.91%", "ICD-9")
|Diagnosis( "404%", "ICD-9")
|Diagnosis( "404.0%", "ICD-9")
|Diagnosis( "404.00%", "ICD-9")
|Diagnosis( "404.01%", "ICD-9")
|Diagnosis( "404.02%", "ICD-9")
|Diagnosis( "404.03%", "ICD-9")
|Diagnosis( "404.1%", "ICD-9")
|Diagnosis( "404.10%", "ICD-9")
|Diagnosis( "404.11%", "ICD-9")
|Diagnosis( "404.12%", "ICD-9")
|Diagnosis( "404.13%", "ICD-9")
|Diagnosis( "404.9%", "ICD-9")
|Diagnosis( "404.90%", "ICD-9")
|Diagnosis( "404.91%", "ICD-9")
|Diagnosis( "404.92%", "ICD-9")
|Diagnosis( "404.93%", "ICD-9")
|Diagnosis( "405%", "ICD-9")
|Diagnosis( "405.0%", "ICD-9")
|Diagnosis( "405.01%", "ICD-9")
|Diagnosis( "405.09%", "ICD-9")
|Diagnosis( "405.1%", "ICD-9")
|Diagnosis( "405.11%", "ICD-9")
|Diagnosis( "405.19%", "ICD-9")
|Diagnosis( "405.9%", "ICD-9")
|Diagnosis( "405.91%", "ICD-9")
|Diagnosis( "405.99%", "ICD-9")
|Diagnosis( "437.2%", "ICD-9")
|Diagnosis( "642.0%", "ICD-9")
|Diagnosis( "642.00%", "ICD-9")
|Diagnosis( "642.01%", "ICD-9")
|Diagnosis( "642.02%", "ICD-9")
|Diagnosis( "642.03%", "ICD-9")
|Diagnosis( "642.04%", "ICD-9")
|Diagnosis( "642.1%", "ICD-9")
|Diagnosis( "642.10%", "ICD-9")
|Diagnosis( "642.11%", "ICD-9")
|Diagnosis( "642.12%", "ICD-9")
|Diagnosis( "642.13%", "ICD-9")
|Diagnosis( "642.14%", "ICD-9")
|Diagnosis( "642.2%", "ICD-9")
|Diagnosis( "642.20%", "ICD-9")
|Diagnosis( "642.21%", "ICD-9")
|Diagnosis( "642.22%", "ICD-9")
|Diagnosis( "642.23%", "ICD-9")
|Diagnosis( "642.24%", "ICD-9")
|Diagnosis( "642.90%", "ICD-9")
|Diagnosis( "642.91%", "ICD-9")
|Diagnosis( "642.92%", "ICD-9")
|Diagnosis( "642.94%", "ICD-9")
|Diagnosis( "H35.039%", "ICD-10")
|Diagnosis( "I10%", "ICD-10")
|Diagnosis( "I11.0%", "ICD-10")
|Diagnosis( "I11.9%", "ICD-10")
|Diagnosis( "I12.0%", "ICD-10")
|Diagnosis( "I12.9%", "ICD-10")
|Diagnosis( "I13.0%", "ICD-10")
|Diagnosis( "I13.10%", "ICD-10")
|Diagnosis( "I13.11%", "ICD-10")
|Diagnosis( "I13.2%", "ICD-10")
|Diagnosis( "I15.0%", "ICD-10")
|Diagnosis( "I15.8%", "ICD-10")
|Diagnosis( "I15.9%", "ICD-10")
|Diagnosis( "I16.9%", "ICD-10")
|Diagnosis( "I67.4%", "ICD-10")
|Diagnosis( "O10.011%", "ICD-10")
|Diagnosis( "O10.012%", "ICD-10")
|Diagnosis( "O10.013%", "ICD-10")
|Diagnosis( "O10.019%", "ICD-10")
|Diagnosis( "O10.02%", "ICD-10")
|Diagnosis( "O10.03%", "ICD-10")
|Diagnosis( "O10.111%", "ICD-10")
|Diagnosis( "O10.112%", "ICD-10")
|Diagnosis( "O10.113%", "ICD-10")
|Diagnosis( "O10.119%", "ICD-10")
|Diagnosis( "O10.12%", "ICD-10")
|Diagnosis( "O10.13%", "ICD-10")
|Diagnosis( "O10.211%", "ICD-10")
|Diagnosis( "O10.212%", "ICD-10")
|Diagnosis( "O10.213%", "ICD-10")
|Diagnosis( "O10.219%", "ICD-10")
|Diagnosis( "O10.22%", "ICD-10")
|Diagnosis( "O10.23%", "ICD-10")
|Diagnosis( "O10.311%", "ICD-10")
|Diagnosis( "O10.312%", "ICD-10")
|Diagnosis( "O10.313%", "ICD-10")
|Diagnosis( "O10.319%", "ICD-10")
|Diagnosis( "O10.32%", "ICD-10")
|Diagnosis( "O10.33%", "ICD-10")
|Diagnosis( "O10.411%", "ICD-10")
|Diagnosis( "O10.412%", "ICD-10")
|Diagnosis( "O10.413%", "ICD-10")
|Diagnosis( "O10.419%", "ICD-10")
|Diagnosis( "O10.42%", "ICD-10")
|Diagnosis( "O10.43%", "ICD-10")
|Diagnosis( "O10.911%", "ICD-10")
|Diagnosis( "O10.912%", "ICD-10")
|Diagnosis( "O10.913%", "ICD-10")
|Diagnosis( "O10.919%", "ICD-10")
|Diagnosis( "O10.92%", "ICD-10")
|Diagnosis( "O10.93%", "ICD-10")
|Diagnosis( "O11.1%", "ICD-10")
|Diagnosis( "O11.2%", "ICD-10")
|Diagnosis( "O11.3%", "ICD-10")
|Diagnosis( "O11.9%", "ICD-10")
|Diagnosis( "O16%", "ICD-10")
|Diagnosis( "O16.1%", "ICD-10")
|Diagnosis( "O16.2%", "ICD-10")
|Diagnosis( "O16.3%", "ICD-10")
|Diagnosis( "O16.4%", "ICD-10")
|Diagnosis( "O16.5%", "ICD-10")
|Diagnosis( "O16.9%", "ICD-10")) 
condition_obesity=( 
Diagnosis( "278%", "ICD-9")
|Diagnosis( "278.0%", "ICD-9")
|Diagnosis( "278.00%", "ICD-9")
|Diagnosis( "278.01%", "ICD-9")
|Diagnosis( "278.03%", "ICD-9")
|Diagnosis( "649.10%", "ICD-9")
|Diagnosis( "649.11%", "ICD-9")
|Diagnosis( "649.12%", "ICD-9")
|Diagnosis( "649.13%", "ICD-9")
|Diagnosis( "649.14%", "ICD-9")
|Diagnosis( "V85.30%", "ICD-9")
|Diagnosis( "V85.31%", "ICD-9")
|Diagnosis( "V85.32%", "ICD-9")
|Diagnosis( "V85.33%", "ICD-9")
|Diagnosis( "V85.34%", "ICD-9")
|Diagnosis( "V85.35%", "ICD-9")
|Diagnosis( "V85.36%", "ICD-9")
|Diagnosis( "V85.37%", "ICD-9")
|Diagnosis( "V85.38%", "ICD-9")
|Diagnosis( "V85.39%", "ICD-9")
|Diagnosis( "V85.4%", "ICD-9")
|Diagnosis( "V85.41%", "ICD-9")
|Diagnosis( "V85.42%", "ICD-9")
|Diagnosis( "V85.43%", "ICD-9")
|Diagnosis( "V85.44%", "ICD-9")
|Diagnosis( "V85.45%", "ICD-9")
|Diagnosis( "V85.54%", "ICD-9")
|Diagnosis( "E66.01%", "ICD-10")
|Diagnosis( "E66.2%", "ICD-10")
|Diagnosis( "E66.9%", "ICD-10")
|Diagnosis( "O99.210%", "ICD-10")
|Diagnosis( "O99.211%", "ICD-10")
|Diagnosis( "O99.212%", "ICD-10")
|Diagnosis( "O99.213%", "ICD-10")
|Diagnosis( "O99.214%", "ICD-10")
|Diagnosis( "O99.215%", "ICD-10")
|Diagnosis( "Z68.30%", "ICD-10")
|Diagnosis( "Z68.31%", "ICD-10")
|Diagnosis( "Z68.32%", "ICD-10")
|Diagnosis( "Z68.33%", "ICD-10")
|Diagnosis( "Z68.34%", "ICD-10")
|Diagnosis( "Z68.35%", "ICD-10")
|Diagnosis( "Z68.36%", "ICD-10")
|Diagnosis( "Z68.37%", "ICD-10")
|Diagnosis( "Z68.38%", "ICD-10")
|Diagnosis( "Z68.39%", "ICD-10")
|Diagnosis( "Z68.41%", "ICD-10")
|Diagnosis( "Z68.42%", "ICD-10")
|Diagnosis( "Z68.43%", "ICD-10")
|Diagnosis( "Z68.44%", "ICD-10")
|Diagnosis( "Z68.45%", "ICD-10")
|Diagnosis( "Z68.54%", "ICD-10")) 
condition_peripheral_artery_disease=( 
Diagnosis( "440%", "ICD-9")
|Diagnosis( "440.0%", "ICD-9")
|Diagnosis( "440.1%", "ICD-9")
|Diagnosis( "440.2%", "ICD-9")
|Diagnosis( "440.20%", "ICD-9")
|Diagnosis( "440.21%", "ICD-9")
|Diagnosis( "440.22%", "ICD-9")
|Diagnosis( "440.23%", "ICD-9")
|Diagnosis( "440.24%", "ICD-9")
|Diagnosis( "440.29%", "ICD-9")
|Diagnosis( "440.3%", "ICD-9")
|Diagnosis( "440.30%", "ICD-9")
|Diagnosis( "440.31%", "ICD-9")
|Diagnosis( "440.32%", "ICD-9")
|Diagnosis( "440.4%", "ICD-9")
|Diagnosis( "440.8%", "ICD-9")
|Diagnosis( "440.9%", "ICD-9")
|Diagnosis( "441%", "ICD-9")
|Diagnosis( "441.0%", "ICD-9")
|Diagnosis( "441.00%", "ICD-9")
|Diagnosis( "441.01%", "ICD-9")
|Diagnosis( "441.02%", "ICD-9")
|Diagnosis( "441.03%", "ICD-9")
|Diagnosis( "441.1%", "ICD-9")
|Diagnosis( "441.2%", "ICD-9")
|Diagnosis( "441.3%", "ICD-9")
|Diagnosis( "441.4%", "ICD-9")
|Diagnosis( "441.5%", "ICD-9")
|Diagnosis( "441.6%", "ICD-9")
|Diagnosis( "441.7%", "ICD-9")
|Diagnosis( "441.9%", "ICD-9")
|Diagnosis( "442.0%", "ICD-9")
|Diagnosis( "442.1%", "ICD-9")
|Diagnosis( "442.2%", "ICD-9")
|Diagnosis( "442.3%", "ICD-9")
|Diagnosis( "442.8%", "ICD-9")
|Diagnosis( "442.82%", "ICD-9")
|Diagnosis( "442.83%", "ICD-9")
|Diagnosis( "442.84%", "ICD-9")
|Diagnosis( "442.89%", "ICD-9")
|Diagnosis( "442.9%", "ICD-9")
|Diagnosis( "443.1%", "ICD-9")
|Diagnosis( "443.21%", "ICD-9")
|Diagnosis( "443.22%", "ICD-9")
|Diagnosis( "443.23%", "ICD-9")
|Diagnosis( "443.24%", "ICD-9")
|Diagnosis( "443.29%", "ICD-9")
|Diagnosis( "443.8%", "ICD-9")
|Diagnosis( "443.81%", "ICD-9")
|Diagnosis( "443.82%", "ICD-9")
|Diagnosis( "443.89%", "ICD-9")
|Diagnosis( "443.9%", "ICD-9")
|Diagnosis( "444%", "ICD-9")
|Diagnosis( "444.0%", "ICD-9")
|Diagnosis( "444.01%", "ICD-9")
|Diagnosis( "444.09%", "ICD-9")
|Diagnosis( "444.1%", "ICD-9")
|Diagnosis( "444.2%", "ICD-9")
|Diagnosis( "444.21%", "ICD-9")
|Diagnosis( "444.22%", "ICD-9")
|Diagnosis( "444.8%", "ICD-9")
|Diagnosis( "444.81%", "ICD-9")
|Diagnosis( "444.89%", "ICD-9")
|Diagnosis( "444.9%", "ICD-9")
|Diagnosis( "447.1%", "ICD-9")
|Diagnosis( "557.1%", "ICD-9")
|Diagnosis( "557.9%", "ICD-9")
|Diagnosis( "785.4%", "ICD-9")
|Diagnosis( "V43.4%", "ICD-9")
|Diagnosis( "I70.0%", "ICD-10")
|Diagnosis( "I70.1%", "ICD-10")
|Diagnosis( "I70.209%", "ICD-10")
|Diagnosis( "I70.219%", "ICD-10")
|Diagnosis( "I70.229%", "ICD-10")
|Diagnosis( "I70.25%", "ICD-10")
|Diagnosis( "I70.269%", "ICD-10")
|Diagnosis( "I70.299%", "ICD-10")
|Diagnosis( "I70.399%", "ICD-10")
|Diagnosis( "I70.499%", "ICD-10")
|Diagnosis( "I70.599%", "ICD-10")
|Diagnosis( "I70.8%", "ICD-10")
|Diagnosis( "I70.90%", "ICD-10")
|Diagnosis( "I70.91%", "ICD-10")
|Diagnosis( "I70.92%", "ICD-10")
|Diagnosis( "I71.00%", "ICD-10")
|Diagnosis( "I71.01%", "ICD-10")
|Diagnosis( "I71.02%", "ICD-10")
|Diagnosis( "I71.03%", "ICD-10")
|Diagnosis( "I71.1%", "ICD-10")
|Diagnosis( "I71.2%", "ICD-10")
|Diagnosis( "I71.3%", "ICD-10")
|Diagnosis( "I71.4%", "ICD-10")
|Diagnosis( "I71.5%", "ICD-10")
|Diagnosis( "I71.6%", "ICD-10")
|Diagnosis( "I71.8%", "ICD-10")
|Diagnosis( "I71.9%", "ICD-10")
|Diagnosis( "I72.1%", "ICD-10")
|Diagnosis( "I72.2%", "ICD-10")
|Diagnosis( "I72.3%", "ICD-10")
|Diagnosis( "I72.4%", "ICD-10")
|Diagnosis( "I72.8%", "ICD-10")
|Diagnosis( "I72.9%", "ICD-10")
|Diagnosis( "I73.01%", "ICD-10")
|Diagnosis( "I73.1%", "ICD-10")
|Diagnosis( "I73.81%", "ICD-10")
|Diagnosis( "I73.89%", "ICD-10")
|Diagnosis( "I73.9%", "ICD-10")
|Diagnosis( "I74.01%", "ICD-10")
|Diagnosis( "I74.09%", "ICD-10")
|Diagnosis( "I74.11%", "ICD-10")
|Diagnosis( "I74.2%", "ICD-10")
|Diagnosis( "I74.3%", "ICD-10")
|Diagnosis( "I74.5%", "ICD-10")
|Diagnosis( "I74.8%", "ICD-10")
|Diagnosis( "I74.9%", "ICD-10")
|Diagnosis( "I77.1%", "ICD-10")
|Diagnosis( "I77.71%", "ICD-10")
|Diagnosis( "I77.72%", "ICD-10")
|Diagnosis( "I77.73%", "ICD-10")
|Diagnosis( "I77.74%", "ICD-10")
|Diagnosis( "I77.75%", "ICD-10")
|Diagnosis( "I77.76%", "ICD-10")
|Diagnosis( "I77.77%", "ICD-10")
|Diagnosis( "I77.79%", "ICD-10")
|Diagnosis( "I79.0%", "ICD-10")
|Diagnosis( "I79.1%", "ICD-10")
|Diagnosis( "I79.8%", "ICD-10")
|Diagnosis( "I96%", "ICD-10")
|Diagnosis( "K55.8%", "ICD-10")
|Diagnosis( "K55.9%", "ICD-10")
|Diagnosis( "Z95.820%", "ICD-10")
|Diagnosis( "Z95.828%", "ICD-10")) 
condition_pulmonary_hypertension=( 
Diagnosis( "416.0%", "ICD-9")
|Diagnosis( "416.9%", "ICD-9")
|Diagnosis( "I27.0%", "ICD-10")
|Diagnosis( "I27.81%", "ICD-10")
|Diagnosis( "I27.9%", "ICD-10")) 
condition_rheumatic_heart_disease=( 
Diagnosis( "393%", "ICD-9")
|Diagnosis( "394%", "ICD-9")
|Diagnosis( "394.0%", "ICD-9")
|Diagnosis( "394.1%", "ICD-9")
|Diagnosis( "394.2%", "ICD-9")
|Diagnosis( "394.9%", "ICD-9")
|Diagnosis( "395%", "ICD-9")
|Diagnosis( "395.0%", "ICD-9")
|Diagnosis( "395.1%", "ICD-9")
|Diagnosis( "395.2%", "ICD-9")
|Diagnosis( "395.9%", "ICD-9")
|Diagnosis( "396%", "ICD-9")
|Diagnosis( "396.0%", "ICD-9")
|Diagnosis( "396.1%", "ICD-9")
|Diagnosis( "396.2%", "ICD-9")
|Diagnosis( "396.3%", "ICD-9")
|Diagnosis( "396.8%", "ICD-9")
|Diagnosis( "396.9%", "ICD-9")
|Diagnosis( "397%", "ICD-9")
|Diagnosis( "397.0%", "ICD-9")
|Diagnosis( "397.1%", "ICD-9")
|Diagnosis( "397.9%", "ICD-9")
|Diagnosis( "398%", "ICD-9")
|Diagnosis( "398.0%", "ICD-9")
|Diagnosis( "398.9%", "ICD-9")
|Diagnosis( "398.90%", "ICD-9")
|Diagnosis( "398.91%", "ICD-9")
|Diagnosis( "398.99%", "ICD-9")
|Diagnosis( "I05.0%", "ICD-10")
|Diagnosis( "I05.1%", "ICD-10")
|Diagnosis( "I05.2%", "ICD-10")
|Diagnosis( "I05.8%", "ICD-10")
|Diagnosis( "I06.0%", "ICD-10")
|Diagnosis( "I06.1%", "ICD-10")
|Diagnosis( "I06.2%", "ICD-10")
|Diagnosis( "I06.8%", "ICD-10")
|Diagnosis( "I06.9%", "ICD-10")
|Diagnosis( "I07.1%", "ICD-10")
|Diagnosis( "I07.2%", "ICD-10")
|Diagnosis( "I07.8%", "ICD-10")
|Diagnosis( "I08.0%", "ICD-10")
|Diagnosis( "I08.8%", "ICD-10")
|Diagnosis( "I08.9%", "ICD-10")
|Diagnosis( "I09.0%", "ICD-10")
|Diagnosis( "I09.1%", "ICD-10")
|Diagnosis( "I09.2%", "ICD-10")
|Diagnosis( "I09.81%", "ICD-10")
|Diagnosis( "I09.89%", "ICD-10")
|Diagnosis( "I09.9%", "ICD-10")) 
condition_sleep_apnea=( 
Diagnosis( "327.23%", "ICD-9")
|Diagnosis( "G47.33%", "ICD-10")) 
condition_stroke_broad=( 
Diagnosis( "342%", "ICD-9")
|Diagnosis( "342.00%", "ICD-9")
|Diagnosis( "342.01%", "ICD-9")
|Diagnosis( "342.02%", "ICD-9")
|Diagnosis( "342.10%", "ICD-9")
|Diagnosis( "342.11%", "ICD-9")
|Diagnosis( "342.12%", "ICD-9")
|Diagnosis( "342.80%", "ICD-9")
|Diagnosis( "342.81%", "ICD-9")
|Diagnosis( "342.82%", "ICD-9")
|Diagnosis( "342.90%", "ICD-9")
|Diagnosis( "342.91%", "ICD-9")
|Diagnosis( "342.92%", "ICD-9")
|Diagnosis( "430%", "ICD-9")
|Diagnosis( "431%", "ICD-9")
|Diagnosis( "432.0%", "ICD-9")
|Diagnosis( "432.1%", "ICD-9")
|Diagnosis( "432.9%", "ICD-9")
|Diagnosis( "433.00%", "ICD-9")
|Diagnosis( "433.01%", "ICD-9")
|Diagnosis( "433.10%", "ICD-9")
|Diagnosis( "433.11%", "ICD-9")
|Diagnosis( "433.20%", "ICD-9")
|Diagnosis( "433.21%", "ICD-9")
|Diagnosis( "433.31%", "ICD-9")
|Diagnosis( "433.80%", "ICD-9")
|Diagnosis( "433.81%", "ICD-9")
|Diagnosis( "433.90%", "ICD-9")
|Diagnosis( "433.91%", "ICD-9")
|Diagnosis( "434.00%", "ICD-9")
|Diagnosis( "434.01%", "ICD-9")
|Diagnosis( "434.10%", "ICD-9")
|Diagnosis( "434.11%", "ICD-9")
|Diagnosis( "434.90%", "ICD-9")
|Diagnosis( "434.91%", "ICD-9")
|Diagnosis( "435.0%", "ICD-9")
|Diagnosis( "435.1%", "ICD-9")
|Diagnosis( "435.2%", "ICD-9")
|Diagnosis( "435.3%", "ICD-9")
|Diagnosis( "435.8%", "ICD-9")
|Diagnosis( "435.9%", "ICD-9")
|Diagnosis( "436%", "ICD-9")
|Diagnosis( "437.0%", "ICD-9")
|Diagnosis( "437.1%", "ICD-9")
|Diagnosis( "437.2%", "ICD-9")
|Diagnosis( "437.3%", "ICD-9")
|Diagnosis( "437.4%", "ICD-9")
|Diagnosis( "437.5%", "ICD-9")
|Diagnosis( "437.6%", "ICD-9")
|Diagnosis( "437.7%", "ICD-9")
|Diagnosis( "437.8%", "ICD-9")
|Diagnosis( "437.9%", "ICD-9")
|Diagnosis( "438.0%", "ICD-9")
|Diagnosis( "438.10%", "ICD-9")
|Diagnosis( "438.11%", "ICD-9")
|Diagnosis( "438.12%", "ICD-9")
|Diagnosis( "438.13%", "ICD-9")
|Diagnosis( "438.14%", "ICD-9")
|Diagnosis( "438.19%", "ICD-9")
|Diagnosis( "438.20%", "ICD-9")
|Diagnosis( "438.21%", "ICD-9")
|Diagnosis( "438.22%", "ICD-9")
|Diagnosis( "438.30%", "ICD-9")
|Diagnosis( "438.31%", "ICD-9")
|Diagnosis( "438.32%", "ICD-9")
|Diagnosis( "438.40%", "ICD-9")
|Diagnosis( "438.41%", "ICD-9")
|Diagnosis( "438.42%", "ICD-9")
|Diagnosis( "438.50%", "ICD-9")
|Diagnosis( "438.51%", "ICD-9")
|Diagnosis( "438.52%", "ICD-9")
|Diagnosis( "438.53%", "ICD-9")
|Diagnosis( "438.6%", "ICD-9")
|Diagnosis( "438.7%", "ICD-9")
|Diagnosis( "438.81%", "ICD-9")
|Diagnosis( "438.82%", "ICD-9")
|Diagnosis( "438.83%", "ICD-9")
|Diagnosis( "438.84%", "ICD-9")
|Diagnosis( "438.89%", "ICD-9")
|Diagnosis( "438.9%", "ICD-9")
|Diagnosis( "997.02%", "ICD-9")
|Diagnosis( "G45.0%", "ICD-10")
|Diagnosis( "G45.1%", "ICD-10")
|Diagnosis( "G45.4%", "ICD-10")
|Diagnosis( "G45.8%", "ICD-10")
|Diagnosis( "G45.9%", "ICD-10")
|Diagnosis( "G46.3%", "ICD-10")
|Diagnosis( "G46.4%", "ICD-10")
|Diagnosis( "G81.00%", "ICD-10")
|Diagnosis( "G81.01%", "ICD-10")
|Diagnosis( "G81.02%", "ICD-10")
|Diagnosis( "G81.03%", "ICD-10")
|Diagnosis( "G81.04%", "ICD-10")
|Diagnosis( "G81.10%", "ICD-10")
|Diagnosis( "G81.11%", "ICD-10")
|Diagnosis( "G81.12%", "ICD-10")
|Diagnosis( "G81.13%", "ICD-10")
|Diagnosis( "G81.14%", "ICD-10")
|Diagnosis( "G81.90%", "ICD-10")
|Diagnosis( "G81.91%", "ICD-10")
|Diagnosis( "G81.92%", "ICD-10")
|Diagnosis( "G81.93%", "ICD-10")
|Diagnosis( "G81.94%", "ICD-10")
|Diagnosis( "I60%", "ICD-10")
|Diagnosis( "I60.0%", "ICD-10")
|Diagnosis( "I60.00%", "ICD-10")
|Diagnosis( "I60.01%", "ICD-10")
|Diagnosis( "I60.02%", "ICD-10")
|Diagnosis( "I60.1%", "ICD-10")
|Diagnosis( "I60.10%", "ICD-10")
|Diagnosis( "I60.11%", "ICD-10")
|Diagnosis( "I60.12%", "ICD-10")
|Diagnosis( "I60.2%", "ICD-10")
|Diagnosis( "I60.3%", "ICD-10")
|Diagnosis( "I60.30%", "ICD-10")
|Diagnosis( "I60.31%", "ICD-10")
|Diagnosis( "I60.32%", "ICD-10")
|Diagnosis( "I60.4%", "ICD-10")
|Diagnosis( "I60.5%", "ICD-10")
|Diagnosis( "I60.50%", "ICD-10")
|Diagnosis( "I60.51%", "ICD-10")
|Diagnosis( "I60.52%", "ICD-10")
|Diagnosis( "I60.6%", "ICD-10")
|Diagnosis( "I60.7%", "ICD-10")
|Diagnosis( "I60.8%", "ICD-10")
|Diagnosis( "I60.9%", "ICD-10")
|Diagnosis( "I61%", "ICD-10")
|Diagnosis( "I61.0%", "ICD-10")
|Diagnosis( "I61.1%", "ICD-10")
|Diagnosis( "I61.2%", "ICD-10")
|Diagnosis( "I61.3%", "ICD-10")
|Diagnosis( "I61.4%", "ICD-10")
|Diagnosis( "I61.5%", "ICD-10")
|Diagnosis( "I61.6%", "ICD-10")
|Diagnosis( "I61.8%", "ICD-10")
|Diagnosis( "I61.9%", "ICD-10")
|Diagnosis( "I62%", "ICD-10")
|Diagnosis( "I62.0%", "ICD-10")
|Diagnosis( "I62.00%", "ICD-10")
|Diagnosis( "I62.01%", "ICD-10")
|Diagnosis( "I62.02%", "ICD-10")
|Diagnosis( "I62.03%", "ICD-10")
|Diagnosis( "I62.1%", "ICD-10")
|Diagnosis( "I62.9%", "ICD-10")
|Diagnosis( "I63%", "ICD-10")
|Diagnosis( "I63.0%", "ICD-10")
|Diagnosis( "I63.00%", "ICD-10")
|Diagnosis( "I63.01%", "ICD-10")
|Diagnosis( "I63.011%", "ICD-10")
|Diagnosis( "I63.012%", "ICD-10")
|Diagnosis( "I63.013%", "ICD-10")
|Diagnosis( "I63.019%", "ICD-10")
|Diagnosis( "I63.02%", "ICD-10")
|Diagnosis( "I63.03%", "ICD-10")
|Diagnosis( "I63.031%", "ICD-10")
|Diagnosis( "I63.032%", "ICD-10")
|Diagnosis( "I63.033%", "ICD-10")
|Diagnosis( "I63.039%", "ICD-10")
|Diagnosis( "I63.09%", "ICD-10")
|Diagnosis( "I63.1%", "ICD-10")
|Diagnosis( "I63.10%", "ICD-10")
|Diagnosis( "I63.11%", "ICD-10")
|Diagnosis( "I63.111%", "ICD-10")
|Diagnosis( "I63.112%", "ICD-10")
|Diagnosis( "I63.113%", "ICD-10")
|Diagnosis( "I63.119%", "ICD-10")
|Diagnosis( "I63.12%", "ICD-10")
|Diagnosis( "I63.13%", "ICD-10")
|Diagnosis( "I63.131%", "ICD-10")
|Diagnosis( "I63.132%", "ICD-10")
|Diagnosis( "I63.133%", "ICD-10")
|Diagnosis( "I63.139%", "ICD-10")
|Diagnosis( "I63.19%", "ICD-10")
|Diagnosis( "I63.2%", "ICD-10")
|Diagnosis( "I63.20%", "ICD-10")
|Diagnosis( "I63.21%", "ICD-10")
|Diagnosis( "I63.211%", "ICD-10")
|Diagnosis( "I63.212%", "ICD-10")
|Diagnosis( "I63.213%", "ICD-10")
|Diagnosis( "I63.219%", "ICD-10")
|Diagnosis( "I63.22%", "ICD-10")
|Diagnosis( "I63.23%", "ICD-10")
|Diagnosis( "I63.231%", "ICD-10")
|Diagnosis( "I63.232%", "ICD-10")
|Diagnosis( "I63.233%", "ICD-10")
|Diagnosis( "I63.239%", "ICD-10")
|Diagnosis( "I63.29%", "ICD-10")
|Diagnosis( "I63.3%", "ICD-10")
|Diagnosis( "I63.30%", "ICD-10")
|Diagnosis( "I63.31%", "ICD-10")
|Diagnosis( "I63.311%", "ICD-10")
|Diagnosis( "I63.312%", "ICD-10")
|Diagnosis( "I63.313%", "ICD-10")
|Diagnosis( "I63.319%", "ICD-10")
|Diagnosis( "I63.32%", "ICD-10")
|Diagnosis( "I63.321%", "ICD-10")
|Diagnosis( "I63.322%", "ICD-10")
|Diagnosis( "I63.323%", "ICD-10")
|Diagnosis( "I63.329%", "ICD-10")
|Diagnosis( "I63.33%", "ICD-10")
|Diagnosis( "I63.331%", "ICD-10")
|Diagnosis( "I63.332%", "ICD-10")
|Diagnosis( "I63.333%", "ICD-10")
|Diagnosis( "I63.339%", "ICD-10")
|Diagnosis( "I63.34%", "ICD-10")
|Diagnosis( "I63.341%", "ICD-10")
|Diagnosis( "I63.342%", "ICD-10")
|Diagnosis( "I63.343%", "ICD-10")
|Diagnosis( "I63.349%", "ICD-10")
|Diagnosis( "I63.39%", "ICD-10")
|Diagnosis( "I63.4%", "ICD-10")
|Diagnosis( "I63.40%", "ICD-10")
|Diagnosis( "I63.41%", "ICD-10")
|Diagnosis( "I63.411%", "ICD-10")
|Diagnosis( "I63.412%", "ICD-10")
|Diagnosis( "I63.413%", "ICD-10")
|Diagnosis( "I63.419%", "ICD-10")
|Diagnosis( "I63.42%", "ICD-10")
|Diagnosis( "I63.421%", "ICD-10")
|Diagnosis( "I63.422%", "ICD-10")
|Diagnosis( "I63.423%", "ICD-10")
|Diagnosis( "I63.429%", "ICD-10")
|Diagnosis( "I63.43%", "ICD-10")
|Diagnosis( "I63.431%", "ICD-10")
|Diagnosis( "I63.432%", "ICD-10")
|Diagnosis( "I63.433%", "ICD-10")
|Diagnosis( "I63.439%", "ICD-10")
|Diagnosis( "I63.44%", "ICD-10")
|Diagnosis( "I63.441%", "ICD-10")
|Diagnosis( "I63.442%", "ICD-10")
|Diagnosis( "I63.443%", "ICD-10")
|Diagnosis( "I63.449%", "ICD-10")
|Diagnosis( "I63.49%", "ICD-10")
|Diagnosis( "I63.5%", "ICD-10")
|Diagnosis( "I63.50%", "ICD-10")
|Diagnosis( "I63.51%", "ICD-10")
|Diagnosis( "I63.511%", "ICD-10")
|Diagnosis( "I63.512%", "ICD-10")
|Diagnosis( "I63.513%", "ICD-10")
|Diagnosis( "I63.519%", "ICD-10")
|Diagnosis( "I63.52%", "ICD-10")
|Diagnosis( "I63.521%", "ICD-10")
|Diagnosis( "I63.522%", "ICD-10")
|Diagnosis( "I63.523%", "ICD-10")
|Diagnosis( "I63.529%", "ICD-10")
|Diagnosis( "I63.53%", "ICD-10")
|Diagnosis( "I63.531%", "ICD-10")
|Diagnosis( "I63.532%", "ICD-10")
|Diagnosis( "I63.533%", "ICD-10")
|Diagnosis( "I63.539%", "ICD-10")
|Diagnosis( "I63.54%", "ICD-10")
|Diagnosis( "I63.541%", "ICD-10")
|Diagnosis( "I63.542%", "ICD-10")
|Diagnosis( "I63.543%", "ICD-10")
|Diagnosis( "I63.549%", "ICD-10")
|Diagnosis( "I63.59%", "ICD-10")
|Diagnosis( "I63.6%", "ICD-10")
|Diagnosis( "I63.8%", "ICD-10")
|Diagnosis( "I63.81%", "ICD-10")
|Diagnosis( "I63.89%", "ICD-10")
|Diagnosis( "I63.9%", "ICD-10")
|Diagnosis( "I65.09%", "ICD-10")
|Diagnosis( "I65.1%", "ICD-10")
|Diagnosis( "I65.29%", "ICD-10")
|Diagnosis( "I65.8%", "ICD-10")
|Diagnosis( "I65.9%", "ICD-10")
|Diagnosis( "I66%", "ICD-10")
|Diagnosis( "I66.0%", "ICD-10")
|Diagnosis( "I66.01%", "ICD-10")
|Diagnosis( "I66.02%", "ICD-10")
|Diagnosis( "I66.03%", "ICD-10")
|Diagnosis( "I66.09%", "ICD-10")
|Diagnosis( "I66.1%", "ICD-10")
|Diagnosis( "I66.11%", "ICD-10")
|Diagnosis( "I66.12%", "ICD-10")
|Diagnosis( "I66.13%", "ICD-10")
|Diagnosis( "I66.19%", "ICD-10")
|Diagnosis( "I66.2%", "ICD-10")
|Diagnosis( "I66.21%", "ICD-10")
|Diagnosis( "I66.22%", "ICD-10")
|Diagnosis( "I66.23%", "ICD-10")
|Diagnosis( "I66.29%", "ICD-10")
|Diagnosis( "I66.3%", "ICD-10")
|Diagnosis( "I66.8%", "ICD-10")
|Diagnosis( "I66.9%", "ICD-10")
|Diagnosis( "I67.1%", "ICD-10")
|Diagnosis( "I67.2%", "ICD-10")
|Diagnosis( "I67.4%", "ICD-10")
|Diagnosis( "I67.5%", "ICD-10")
|Diagnosis( "I67.6%", "ICD-10")
|Diagnosis( "I67.7%", "ICD-10")
|Diagnosis( "I67.81%", "ICD-10")
|Diagnosis( "I67.82%", "ICD-10")
|Diagnosis( "I67.848%", "ICD-10")
|Diagnosis( "I67.89%", "ICD-10")
|Diagnosis( "I67.9%", "ICD-10")
|Diagnosis( "I69.898%", "ICD-10")
|Diagnosis( "I69.90%", "ICD-10")
|Diagnosis( "I69.910%", "ICD-10")
|Diagnosis( "I69.911%", "ICD-10")
|Diagnosis( "I69.912%", "ICD-10")
|Diagnosis( "I69.913%", "ICD-10")
|Diagnosis( "I69.914%", "ICD-10")
|Diagnosis( "I69.915%", "ICD-10")
|Diagnosis( "I69.918%", "ICD-10")
|Diagnosis( "I69.919%", "ICD-10")
|Diagnosis( "I69.920%", "ICD-10")
|Diagnosis( "I69.921%", "ICD-10")
|Diagnosis( "I69.922%", "ICD-10")
|Diagnosis( "I69.923%", "ICD-10")
|Diagnosis( "I69.928%", "ICD-10")
|Diagnosis( "I69.931%", "ICD-10")
|Diagnosis( "I69.932%", "ICD-10")
|Diagnosis( "I69.933%", "ICD-10")
|Diagnosis( "I69.934%", "ICD-10")
|Diagnosis( "I69.939%", "ICD-10")
|Diagnosis( "I69.941%", "ICD-10")
|Diagnosis( "I69.942%", "ICD-10")
|Diagnosis( "I69.943%", "ICD-10")
|Diagnosis( "I69.944%", "ICD-10")
|Diagnosis( "I69.949%", "ICD-10")
|Diagnosis( "I69.951%", "ICD-10")
|Diagnosis( "I69.952%", "ICD-10")
|Diagnosis( "I69.953%", "ICD-10")
|Diagnosis( "I69.954%", "ICD-10")
|Diagnosis( "I69.959%", "ICD-10")
|Diagnosis( "I69.961%", "ICD-10")
|Diagnosis( "I69.962%", "ICD-10")
|Diagnosis( "I69.963%", "ICD-10")
|Diagnosis( "I69.964%", "ICD-10")
|Diagnosis( "I69.965%", "ICD-10")
|Diagnosis( "I69.969%", "ICD-10")
|Diagnosis( "I69.990%", "ICD-10")
|Diagnosis( "I69.991%", "ICD-10")
|Diagnosis( "I69.992%", "ICD-10")
|Diagnosis( "I69.993%", "ICD-10")
|Diagnosis( "I69.998%", "ICD-10")
|Diagnosis( "I97.811%", "ICD-10")
|Diagnosis( "I97.821%", "ICD-10")) 
condition_stroke_hemorrhagic=( 
Diagnosis( "430%", "ICD-9")
|Diagnosis( "431%", "ICD-9")
|Diagnosis( "432%", "ICD-9")
|Diagnosis( "432.0%", "ICD-9")
|Diagnosis( "432.1%", "ICD-9")
|Diagnosis( "432.9%", "ICD-9")
|Diagnosis( "433%", "ICD-9")
|Diagnosis( "433.0%", "ICD-9")
|Diagnosis( "433.00%", "ICD-9")
|Diagnosis( "433.01%", "ICD-9")
|Diagnosis( "433.1%", "ICD-9")
|Diagnosis( "433.10%", "ICD-9")
|Diagnosis( "433.11%", "ICD-9")
|Diagnosis( "433.2%", "ICD-9")
|Diagnosis( "433.20%", "ICD-9")
|Diagnosis( "433.21%", "ICD-9")
|Diagnosis( "433.3%", "ICD-9")
|Diagnosis( "433.30%", "ICD-9")
|Diagnosis( "433.31%", "ICD-9")
|Diagnosis( "433.8%", "ICD-9")
|Diagnosis( "433.80%", "ICD-9")
|Diagnosis( "433.81%", "ICD-9")
|Diagnosis( "433.9%", "ICD-9")
|Diagnosis( "433.90%", "ICD-9")
|Diagnosis( "433.91%", "ICD-9")
|Diagnosis( "I60.9%", "ICD-10")
|Diagnosis( "I61%", "ICD-10")
|Diagnosis( "I61.0%", "ICD-10")
|Diagnosis( "I61.1%", "ICD-10")
|Diagnosis( "I61.2%", "ICD-10")
|Diagnosis( "I61.3%", "ICD-10")
|Diagnosis( "I61.4%", "ICD-10")
|Diagnosis( "I61.5%", "ICD-10")
|Diagnosis( "I61.6%", "ICD-10")
|Diagnosis( "I61.8%", "ICD-10")
|Diagnosis( "I61.9%", "ICD-10")
|Diagnosis( "I62.00%", "ICD-10")
|Diagnosis( "I62.1%", "ICD-10")
|Diagnosis( "I62.9%", "ICD-10")
|Diagnosis( "I63.019%", "ICD-10")
|Diagnosis( "I63.119%", "ICD-10")
|Diagnosis( "I63.139%", "ICD-10")
|Diagnosis( "I63.20%", "ICD-10")
|Diagnosis( "I63.219%", "ICD-10")
|Diagnosis( "I63.22%", "ICD-10")
|Diagnosis( "I63.239%", "ICD-10")
|Diagnosis( "I63.59%", "ICD-10")
|Diagnosis( "I65.09%", "ICD-10")
|Diagnosis( "I65.1%", "ICD-10")
|Diagnosis( "I65.29%", "ICD-10")
|Diagnosis( "I65.8%", "ICD-10")
|Diagnosis( "I65.9%", "ICD-10")) 
condition_stroke_ischemic=( 
Diagnosis( "433%", "ICD-9")
|Diagnosis( "433.0%", "ICD-9")
|Diagnosis( "433.00%", "ICD-9")
|Diagnosis( "433.01%", "ICD-9")
|Diagnosis( "433.1%", "ICD-9")
|Diagnosis( "433.10%", "ICD-9")
|Diagnosis( "433.11%", "ICD-9")
|Diagnosis( "433.2%", "ICD-9")
|Diagnosis( "433.20%", "ICD-9")
|Diagnosis( "433.21%", "ICD-9")
|Diagnosis( "433.3%", "ICD-9")
|Diagnosis( "433.30%", "ICD-9")
|Diagnosis( "433.31%", "ICD-9")
|Diagnosis( "433.8%", "ICD-9")
|Diagnosis( "433.80%", "ICD-9")
|Diagnosis( "433.81%", "ICD-9")
|Diagnosis( "433.9%", "ICD-9")
|Diagnosis( "433.90%", "ICD-9")
|Diagnosis( "433.91%", "ICD-9")
|Diagnosis( "434%", "ICD-9")
|Diagnosis( "434.0%", "ICD-9")
|Diagnosis( "434.00%", "ICD-9")
|Diagnosis( "434.01%", "ICD-9")
|Diagnosis( "434.1%", "ICD-9")
|Diagnosis( "434.10%", "ICD-9")
|Diagnosis( "434.11%", "ICD-9")
|Diagnosis( "434.9%", "ICD-9")
|Diagnosis( "434.90%", "ICD-9")
|Diagnosis( "434.91%", "ICD-9")
|Diagnosis( "435%", "ICD-9")
|Diagnosis( "435.0%", "ICD-9")
|Diagnosis( "435.1%", "ICD-9")
|Diagnosis( "435.2%", "ICD-9")
|Diagnosis( "435.3%", "ICD-9")
|Diagnosis( "435.8%", "ICD-9")
|Diagnosis( "435.9%", "ICD-9")
|Diagnosis( "436%", "ICD-9")
|Diagnosis( "437%", "ICD-9")
|Diagnosis( "437.0%", "ICD-9")
|Diagnosis( "437.1%", "ICD-9")
|Diagnosis( "437.2%", "ICD-9")
|Diagnosis( "437.3%", "ICD-9")
|Diagnosis( "437.4%", "ICD-9")
|Diagnosis( "437.5%", "ICD-9")
|Diagnosis( "437.6%", "ICD-9")
|Diagnosis( "437.7%", "ICD-9")
|Diagnosis( "437.8%", "ICD-9")
|Diagnosis( "437.9%", "ICD-9")
|Diagnosis( "I63.00%", "ICD-10")
|Diagnosis( "I63.011%", "ICD-10")
|Diagnosis( "I63.012%", "ICD-10")
|Diagnosis( "I63.013%", "ICD-10")
|Diagnosis( "I63.019%", "ICD-10")
|Diagnosis( "I63.02%", "ICD-10")
|Diagnosis( "I63.031%", "ICD-10")
|Diagnosis( "I63.032%", "ICD-10")
|Diagnosis( "I63.033%", "ICD-10")
|Diagnosis( "I63.039%", "ICD-10")
|Diagnosis( "I63.09%", "ICD-10")
|Diagnosis( "I63.10%", "ICD-10")
|Diagnosis( "I63.111%", "ICD-10")
|Diagnosis( "I63.112%", "ICD-10")
|Diagnosis( "I63.113%", "ICD-10")
|Diagnosis( "I63.119%", "ICD-10")
|Diagnosis( "I63.12%", "ICD-10")
|Diagnosis( "I63.131%", "ICD-10")
|Diagnosis( "I63.132%", "ICD-10")
|Diagnosis( "I63.133%", "ICD-10")
|Diagnosis( "I63.139%", "ICD-10")
|Diagnosis( "I63.19%", "ICD-10")
|Diagnosis( "I63.20%", "ICD-10")
|Diagnosis( "I63.211%", "ICD-10")
|Diagnosis( "I63.212%", "ICD-10")
|Diagnosis( "I63.213%", "ICD-10")
|Diagnosis( "I63.219%", "ICD-10")
|Diagnosis( "I63.22%", "ICD-10")
|Diagnosis( "I63.231%", "ICD-10")
|Diagnosis( "I63.232%", "ICD-10")
|Diagnosis( "I63.233%", "ICD-10")
|Diagnosis( "I63.239%", "ICD-10")
|Diagnosis( "I63.29%", "ICD-10")
|Diagnosis( "I63.30%", "ICD-10")
|Diagnosis( "I63.311%", "ICD-10")
|Diagnosis( "I63.312%", "ICD-10")
|Diagnosis( "I63.313%", "ICD-10")
|Diagnosis( "I63.319%", "ICD-10")
|Diagnosis( "I63.321%", "ICD-10")
|Diagnosis( "I63.322%", "ICD-10")
|Diagnosis( "I63.323%", "ICD-10")
|Diagnosis( "I63.329%", "ICD-10")
|Diagnosis( "I63.331%", "ICD-10")
|Diagnosis( "I63.332%", "ICD-10")
|Diagnosis( "I63.333%", "ICD-10")
|Diagnosis( "I63.339%", "ICD-10")
|Diagnosis( "I63.341%", "ICD-10")
|Diagnosis( "I63.342%", "ICD-10")
|Diagnosis( "I63.343%", "ICD-10")
|Diagnosis( "I63.349%", "ICD-10")
|Diagnosis( "I63.39%", "ICD-10")
|Diagnosis( "I63.40%", "ICD-10")
|Diagnosis( "I63.411%", "ICD-10")
|Diagnosis( "I63.412%", "ICD-10")
|Diagnosis( "I63.413%", "ICD-10")
|Diagnosis( "I63.419%", "ICD-10")
|Diagnosis( "I63.421%", "ICD-10")
|Diagnosis( "I63.422%", "ICD-10")
|Diagnosis( "I63.423%", "ICD-10")
|Diagnosis( "I63.429%", "ICD-10")
|Diagnosis( "I63.431%", "ICD-10")
|Diagnosis( "I63.432%", "ICD-10")
|Diagnosis( "I63.433%", "ICD-10")
|Diagnosis( "I63.439%", "ICD-10")
|Diagnosis( "I63.441%", "ICD-10")
|Diagnosis( "I63.442%", "ICD-10")
|Diagnosis( "I63.443%", "ICD-10")
|Diagnosis( "I63.449%", "ICD-10")
|Diagnosis( "I63.49%", "ICD-10")
|Diagnosis( "I63.50%", "ICD-10")
|Diagnosis( "I63.511%", "ICD-10")
|Diagnosis( "I63.512%", "ICD-10")
|Diagnosis( "I63.513%", "ICD-10")
|Diagnosis( "I63.519%", "ICD-10")
|Diagnosis( "I63.521%", "ICD-10")
|Diagnosis( "I63.522%", "ICD-10")
|Diagnosis( "I63.523%", "ICD-10")
|Diagnosis( "I63.529%", "ICD-10")
|Diagnosis( "I63.531%", "ICD-10")
|Diagnosis( "I63.532%", "ICD-10")
|Diagnosis( "I63.533%", "ICD-10")
|Diagnosis( "I63.539%", "ICD-10")
|Diagnosis( "I63.541%", "ICD-10")
|Diagnosis( "I63.542%", "ICD-10")
|Diagnosis( "I63.543%", "ICD-10")
|Diagnosis( "I63.549%", "ICD-10")
|Diagnosis( "I63.59%", "ICD-10")
|Diagnosis( "I63.6%", "ICD-10")
|Diagnosis( "I63.8%", "ICD-10")
|Diagnosis( "I63.9%", "ICD-10")
|Diagnosis( "I65.01%", "ICD-10")
|Diagnosis( "I65.02%", "ICD-10")
|Diagnosis( "I65.03%", "ICD-10")
|Diagnosis( "I65.09%", "ICD-10")
|Diagnosis( "I65.1%", "ICD-10")
|Diagnosis( "I65.21%", "ICD-10")
|Diagnosis( "I65.22%", "ICD-10")
|Diagnosis( "I65.23%", "ICD-10")
|Diagnosis( "I65.29%", "ICD-10")
|Diagnosis( "I65.8%", "ICD-10")
|Diagnosis( "I65.9%", "ICD-10")
|Diagnosis( "I66.01%", "ICD-10")
|Diagnosis( "I66.02%", "ICD-10")
|Diagnosis( "I66.03%", "ICD-10")
|Diagnosis( "I66.09%", "ICD-10")
|Diagnosis( "I66.11%", "ICD-10")
|Diagnosis( "I66.12%", "ICD-10")
|Diagnosis( "I66.13%", "ICD-10")
|Diagnosis( "I66.19%", "ICD-10")
|Diagnosis( "I66.21%", "ICD-10")
|Diagnosis( "I66.22%", "ICD-10")
|Diagnosis( "I66.23%", "ICD-10")
|Diagnosis( "I66.29%", "ICD-10")
|Diagnosis( "I66.3%", "ICD-10")
|Diagnosis( "I66.8%", "ICD-10")
|Diagnosis( "I66.9%", "ICD-10")
|Diagnosis( "I67.1%", "ICD-10")
|Diagnosis( "I67.2%", "ICD-10")
|Diagnosis( "I67.4%", "ICD-10")
|Diagnosis( "I67.5%", "ICD-10")
|Diagnosis( "I67.6%", "ICD-10")
|Diagnosis( "I67.7%", "ICD-10")
|Diagnosis( "I67.81%", "ICD-10")
|Diagnosis( "I67.82%", "ICD-10")
|Diagnosis( "I67.841%", "ICD-10")
|Diagnosis( "I67.848%", "ICD-10")
|Diagnosis( "I67.89%", "ICD-10")
|Diagnosis( "I67.9%", "ICD-10")
|Diagnosis( "G45.0%", "ICD-10")
|Diagnosis( "G45.1%", "ICD-10")
|Diagnosis( "G45.4%", "ICD-10")
|Diagnosis( "G45.8%", "ICD-10")
|Diagnosis( "G45.9%", "ICD-10")) 
condition_transient_ischemic_attack=( 
Diagnosis( "435.0%", "ICD-9")
|Diagnosis( "435.1%", "ICD-9")
|Diagnosis( "435.2%", "ICD-9")
|Diagnosis( "435.3%", "ICD-9")
|Diagnosis( "435.8%", "ICD-9")
|Diagnosis( "435.9%", "ICD-9")
|Diagnosis( "437.7%", "ICD-9")
|Diagnosis( "G45%", "ICD-10")
|Diagnosis( "G45.0%", "ICD-10")
|Diagnosis( "G45.1%", "ICD-10")
|Diagnosis( "G45.2%", "ICD-10")
|Diagnosis( "G45.3%", "ICD-10")
|Diagnosis( "G45.4%", "ICD-10")
|Diagnosis( "G45.8%", "ICD-10")
|Diagnosis( "G45.9%", "ICD-10")) 
condition_valve_disorder=( 
Diagnosis( "424.0%", "ICD-9")
|Diagnosis( "424.1%", "ICD-9")
|Diagnosis( "424.2%", "ICD-9")
|Diagnosis( "424.3%", "ICD-9")
|Diagnosis( "I34%", "ICD-10")
|Diagnosis( "I34.0%", "ICD-10")
|Diagnosis( "I34.1%", "ICD-10")
|Diagnosis( "I34.2%", "ICD-10")
|Diagnosis( "I34.8%", "ICD-10")
|Diagnosis( "I34.9%", "ICD-10")
|Diagnosis( "I35%", "ICD-10")
|Diagnosis( "I35.0%", "ICD-10")
|Diagnosis( "I35.1%", "ICD-10")
|Diagnosis( "I35.2%", "ICD-10")
|Diagnosis( "I35.8%", "ICD-10")
|Diagnosis( "I35.9%", "ICD-10")
|Diagnosis( "I36%", "ICD-10")
|Diagnosis( "I36.0%", "ICD-10")
|Diagnosis( "I36.1%", "ICD-10")
|Diagnosis( "I36.2%", "ICD-10")
|Diagnosis( "I36.8%", "ICD-10")
|Diagnosis( "I36.9%", "ICD-10")
|Diagnosis( "I37%", "ICD-10")
|Diagnosis( "I37.0%", "ICD-10")
|Diagnosis( "I37.1%", "ICD-10")
|Diagnosis( "I37.2%", "ICD-10")
|Diagnosis( "I37.8%", "ICD-10")
|Diagnosis( "I37.9%", "ICD-10")) 


In [ ]:
#Naming of the Features
features=[
[condition_vascular_cognitive_impairment,"vascular_cognitive_impairment"],
[condition_acute_myocardial_infarction,"acute_myocardial_infarction"],
[condition_anemia,"anemia"],
[condition_angina,"angina"],
[condition_atrial_flutter_fibrillation,"atrial_flutter_fibrillation"],
[condition_cardiomyopathy,"cardiomyopathy"],
[condition_chronic_kidney_disease,"chronic_kidney_disease"],
[condition_COPD,"COPD"],
[condition_coronary_artery_disease,"coronary_artery_disease"],
[condition_depression,"depression"],
[condition_diabetes_mellitus_type_I,"diabetes_mellitus_type_I"],
[condition_diabetes_mellitus_type_II,"diabetes_mellitus_type_II"],
[condition_diabetic_nephropathy,"diabetic_nephropathy"],
[condition_dyspnea,"dyspnea"],
[condition_dysrhythmias,"dysrhythmias"],
[condition_edema,"edema"],
[condition_hyperkalemia,"hyperkalemia"],
[condition_hyperlipidemia,"hyperlipidemia"],
[condition_hypertension,"hypertension"],
[condition_obesity,"obesity"],
[condition_peripheral_artery_disease,"peripheral_artery_disease"],
[condition_pulmonary_hypertension,"pulmonary_hypertension"],
[condition_rheumatic_heart_disease,"rheumatic_heart_disease"],
[condition_sleep_apnea,"sleep_apnea"],
[condition_stroke_broad,"stroke_broad"],
[condition_stroke_hemorrhagic,"stroke_hemorrhagic"],
[condition_stroke_ischemic,"stroke_ischemic"],
[condition_transient_ischemic_attack,"transient_ischemic_attack"],
[condition_valve_disorder,"valve_disorder"]]


## Drug
These were defined by the Notebook Create_supervised_Features

In [ ]:
condition_oral_diuretics=( 
Drug( "%Loop Diuretics%")
|Drug( "%Thiazide diuretics%")
|Drug( "%Potassium sparing diuretics%")) 

condition_angiotensin_receptor_blockers=( 
Drug( "%Azilsartan%")
|Drug( "%Candesartan%")
|Drug( "%Eprosartan%")
|Drug( "%Irbesartan%")
|Drug( "%Losartan%")
|Drug( "%Olmesartan%")
|Drug( "%Telmisartan%")
|Drug( "%Valsartan%")) 
condition_beta_blockers=( 
Drug( "%Acebutolol%")
|Drug( "%Atenolol%")
|Drug( "%Bisoprolol%")
|Drug( "%Metoprolol%")
|Drug( "%Nadolol%")
|Drug( "%Nebivolol%")
|Drug( "%Propranolol%")) 
condition_entresto=( 
Drug( "%Entresto%")) 
condition_IV_diuretics=( 
Drug( "%Loop Diuretics%")
|Drug( "%Thiazide diuretics%")
|Drug( "%Potassium sparing diuretics%")
|Drug( "%Bumetanide%")
|Drug( "%Ethacrynate sodium%")
|Drug( "%Ethacrynic acid%")
|Drug( "%Furosemide%")
|Drug( "%Furosemide%")
|Drug( "%Furosemide%")
|Drug( "%Torsemide%")) 
condition_long_acting_nitrates=( 
Drug( "%Isosorbide dinitrate%")
|Drug( "%Isosorbide mononitrate%")) 
condition_mineralocorticoid_receptor_anta=( 
Drug( "%Spironolactone%")
|Drug( "%Eplerenone%")) 



In [ ]:
#Naming of the Features
features =[
[condition_oral_diuretics,"oral_diuretics"],
[condition_angiotensin_receptor_blockers,"angiotensin_receptor_blockers"],
[condition_beta_blockers,"beta_blockers"],
[condition_entresto,"entresto"],
[condition_IV_diuretics,"IV_diuretics"],
[condition_long_acting_nitrates,"long_acting_nitrates"],
[condition_mineralocorticoid_receptor_anta,"mineralocorticoid_receptor_anta"]
]


In [ ]:
features =[[condition_inpatient,'inpatient_events']]

In [ ]:
#This funtion is extracting the features for a case and control cohort. 
#input: 
#case= Pandas data frame
#control= Pandas data frame
#feature= Array of features, which contains the FIBER Condition and the name for a feature, that should be used in the data frame
#gap_in_days= Gap in days before the exposure of a patient. It has to be an integer
#frequency = type of occurence: EVER, COUNT,WINDOW
def get_features(case, features, frequency):
    # case 
    for feature in features: 
        df_final = get_has_certain_condition(feature[0],case,feature[1], frequency)
        if df_final.empty==False:
            df_final.to_parquet("Cohort/Feature_Extraction/Supervised_ALL_HF/"+feature[1]+".parquet")
    #control
   # for feature in features: 
    #    df_final = get_has_certain_condition(feature[0],control,feature[1],gap_in_days, frequency, "Control")
     #   df_final.to_parquet(df_name+".parquet")
    

In [ ]:
# Reading cohort as dataframe
Case_EF_ICD = pq.read_table('Cohort/Phenotyping/ALL_Matches_1yr_HF_EF_ICD_Notes_Cohort.parquet').to_pandas()
Case_EF_ICD = Case_EF_ICD.rename(columns={'MRN': 'medical_record_number'})
Case_EF_ICD = Case_EF_ICD.rename(columns={'age_in_days': 'age_in_days_term'})
Case_ICD = pq.read_table('Cohort/Phenotyping/ALL_Matches_1yr_HF_ICD_Notes_Cohort.parquet').to_pandas()
Case_ICD = Case_ICD.rename(columns={'MRN': 'medical_record_number'})
Case_ICD = Case_ICD.rename(columns={'age_in_days': 'age_in_days_term'})                                          
#Case_ICD=Case_ICD.set_index('medical_record_number', inplace=False)

In [ ]:
Case_all= pd.concat([Case_EF_ICD, Case_ICD], ignore_index=False, sort =False)

In [ ]:
Case_all

In [ ]:
get_features(Case_all,features,"COUNT")

In [ ]:
# check the output 
test = pq.read_table('Cohort/Feature_Extraction/Supervised_ALL_HF/inpatient_events.parquet').to_pandas()

In [ ]:
test_small=test[['medical_record_number','number_of_occurences_inpatient_events']]
test_small=test_small.set_index('medical_record_number')
test_small.to_parquet('Cohort/Feature_Extraction/Supervised_ALL_HF/inpatient_events_merge.parquet')
test_small

In [ ]:
test.loc[test["has_condition_acute_myocardial_infarction"]==1]